In [265]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split


In [266]:
df_template = pd.DataFrame(columns=["driver_id, driver_ref, q1_position, q2_position, q3_position, circuit, constructor_standing, driver_standing, year, constructor, quali_position"])

In [267]:
drivers = pd.read_csv("archive/drivers.csv")
qualifying = pd.read_csv("archive/qualifying.csv")
constructor_results = pd.read_csv("archive/constructor_results.csv")
race_results = pd.read_csv("archive/results.csv")
constructors = pd.read_csv("archive/constructors.csv")
races = pd.read_csv("archive/races.csv")

In [268]:
driver_ids_list = [1, 4, 853, 854, 822, 830, 815, 847, 849, 846, 817, 839, 840, 20, 852, 842, 841, 8, 832, 844]
drivers_clean = drivers[drivers.driverId.isin(driver_ids_list)].drop(["number", "forename", "surname", "dob", "nationality", "url", "driverRef"], axis=1).reset_index(drop=True)

drivers_clean

,driverId,code
0,1,HAM
1,4,ALO
2,8,RAI
3,20,VET
4,842,GAS
5,815,PER
6,817,RIC
7,822,BOT
8,830,VER
9,832,SAI


In [269]:
qualifying

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
0,1,18,1,1,22,1,1:26.572,1:25.187,1:26.714
1,2,18,9,2,4,2,1:26.103,1:25.315,1:26.869
2,3,18,5,1,23,3,1:25.664,1:25.452,1:27.079
3,4,18,13,6,2,4,1:25.994,1:25.691,1:27.178
4,5,18,2,2,3,5,1:25.960,1:25.518,1:27.236
...,...,...,...,...,...,...,...,...,...
8745,8810,1055,852,213,22,16,1:18.556,NaN,NaN
8746,8811,1055,8,51,7,17,1:18.917,NaN,NaN
8747,8812,1055,854,210,47,18,1:19.117,NaN,NaN
8748,8813,1055,849,3,6,19,1:19.219,NaN,NaN


In [270]:
qualifying = qualifying.drop(["number", "qualifyId", "position"], axis=1)

In [271]:
test1 = qualifying[["raceId", "driverId", "constructorId"]].copy()

test1

,raceId,driverId,constructorId
0,18,1,1
1,18,9,2
2,18,5,1
3,18,13,6
4,18,2,2
...,...,...,...
8745,1055,852,213
8746,1055,8,51
8747,1055,854,210
8748,1055,849,3


In [272]:
q1_df = qualifying.drop(["q2", "q3"], axis=1)
q1_df = q1_df.dropna()
q1_df["q1_position"] = q1_df.sort_values(["q1"]).groupby("raceId").cumcount() + 1

q1_df

,raceId,driverId,constructorId,q1,q1_position
0,18,1,1,1:26.572,10
1,18,9,2,1:26.103,4
2,18,5,1,1:25.664,1
3,18,13,6,1:25.994,3
4,18,2,2,1:25.960,2
...,...,...,...,...,...
8745,1055,852,213,1:18.556,16
8746,1055,8,51,1:18.917,17
8747,1055,854,210,1:19.117,18
8748,1055,849,3,1:19.219,19


In [273]:
q2_df = qualifying.drop(["q1", "q3"], axis=1)
q2_df = q2_df.dropna()
q2_df["q2_position"] = q2_df.sort_values(["q2"]).groupby("raceId").cumcount() + 1


q2_df

,raceId,driverId,constructorId,q2,q2_position
0,18,1,1,1:25.187,1
1,18,9,2,1:25.315,2
2,18,5,1,1:25.452,3
3,18,13,6,1:25.691,5
4,18,2,2,1:25.518,4
...,...,...,...,...,...
8740,1055,840,117,1:17.974,11
8741,1055,842,213,1:17.982,12
8742,1055,20,117,1:18.079,13
8743,1055,841,51,1:18.356,14


In [274]:
q3_df = qualifying.drop(["q1", "q2"], axis=1)
q3_df = q3_df.dropna()
q3_df["q3_position"] = q3_df.sort_values(["q3"]).groupby("raceId").cumcount() + 1


q3_df

,raceId,driverId,constructorId,q3,q3_position
0,18,1,1,1:26.714,1
1,18,9,2,1:26.869,2
2,18,5,1,1:27.079,3
3,18,13,6,1:27.178,4
4,18,2,2,1:27.236,5
...,...,...,...,...,...
8735,1055,832,6,1:17.620,6
8736,1055,817,1,1:17.622,7
8737,1055,815,9,1:17.701,8
8738,1055,846,1,1:18.010,9


In [275]:
test1 = test1.merge(
    q1_df,
    on=["raceId", "driverId", "constructorId"],
    how="left"
)

test1

,raceId,driverId,constructorId,q1,q1_position
0,18,1,1,1:26.572,10.0
1,18,9,2,1:26.103,4.0
2,18,5,1,1:25.664,1.0
3,18,13,6,1:25.994,3.0
4,18,2,2,1:25.960,2.0
...,...,...,...,...,...
8745,1055,852,213,1:18.556,16.0
8746,1055,8,51,1:18.917,17.0
8747,1055,854,210,1:19.117,18.0
8748,1055,849,3,1:19.219,19.0


In [239]:
test1.isna().sum()

raceId             0
driverId           0
constructorId      0
q1               136
q1_position      136
dtype: int64

In [276]:
test1 = test1.merge(
    q2_df,
    on=["raceId", "driverId", "constructorId"],
    how="left"
)

test1

,raceId,driverId,constructorId,q1,q1_position,q2,q2_position
0,18,1,1,1:26.572,10.0,1:25.187,1.0
1,18,9,2,1:26.103,4.0,1:25.315,2.0
2,18,5,1,1:25.664,1.0,1:25.452,3.0
3,18,13,6,1:25.994,3.0,1:25.691,5.0
4,18,2,2,1:25.960,2.0,1:25.518,4.0
...,...,...,...,...,...,...,...
8745,1055,852,213,1:18.556,16.0,NaN,NaN
8746,1055,8,51,1:18.917,17.0,NaN,NaN
8747,1055,854,210,1:19.117,18.0,NaN,NaN
8748,1055,849,3,1:19.219,19.0,NaN,NaN


In [277]:
test1 = test1.merge(
    q3_df,
    on=["raceId", "driverId", "constructorId"],
    how="left"
)

test1

,raceId,driverId,constructorId,q1,q1_position,q2,q2_position,q3,q3_position
0,18,1,1,1:26.572,10.0,1:25.187,1.0,1:26.714,1.0
1,18,9,2,1:26.103,4.0,1:25.315,2.0,1:26.869,2.0
2,18,5,1,1:25.664,1.0,1:25.452,3.0,1:27.079,3.0
3,18,13,6,1:25.994,3.0,1:25.691,5.0,1:27.178,4.0
4,18,2,2,1:25.960,2.0,1:25.518,4.0,1:27.236,5.0
...,...,...,...,...,...,...,...,...,...
8745,1055,852,213,1:18.556,16.0,NaN,NaN,NaN,NaN
8746,1055,8,51,1:18.917,17.0,NaN,NaN,NaN,NaN
8747,1055,854,210,1:19.117,18.0,NaN,NaN,NaN,NaN
8748,1055,849,3,1:19.219,19.0,NaN,NaN,NaN,NaN


In [278]:
cleaned_positions = test1.drop(["q1", "q2", "q3"], axis=1)

cleaned_positions

,raceId,driverId,constructorId,q1_position,q2_position,q3_position
0,18,1,1,10.0,1.0,1.0
1,18,9,2,4.0,2.0,2.0
2,18,5,1,1.0,3.0,3.0
3,18,13,6,3.0,5.0,4.0
4,18,2,2,2.0,4.0,5.0
...,...,...,...,...,...,...
8745,1055,852,213,16.0,NaN,NaN
8746,1055,8,51,17.0,NaN,NaN
8747,1055,854,210,18.0,NaN,NaN
8748,1055,849,3,19.0,NaN,NaN


In [279]:
test_x = cleaned_positions.groupby(["raceId"]).transform(lambda x: x.fillna(x.max() + 1))
test_x["raceId"] = cleaned_positions["raceId"]

In [262]:
test_x.to_csv("./test_x.csv")

In [288]:
test_x.isna().sum()

driverId            0
constructorId       0
q1_position         0
q2_position      2462
q3_position      2598
raceId              0
dtype: int64

In [ ]:
test_x.loc[pd.isna(test_x["q1_position"]), :].index

In [264]:
test_x[2568:2591]

,driverId,constructorId,q1_position,q2_position,q3_position,raceId
2568,71,3,NaN,NaN,NaN,256
2569,14,3,NaN,NaN,NaN,256
2570,30,22,NaN,NaN,NaN,256
2571,77,6,NaN,NaN,NaN,256
2572,55,6,NaN,NaN,NaN,256
2573,49,15,NaN,NaN,NaN,256
2574,22,17,NaN,NaN,NaN,256
2575,65,22,NaN,NaN,NaN,256
2576,56,17,NaN,NaN,NaN,256
2577,87,1,NaN,NaN,NaN,256


In [302]:
def copy_quali_results(df:pd.DataFrame):
    if df["q2_position"].isna().values.all():
        df["q2_position"] = df["q1_position"]
    if df["q3_position"].isna().values.all():
        df["q3_position"] = df["q2_position"]
    return df


In [303]:
cleaned_quali_results = test_x.groupby(["raceId"]).apply(copy_quali_results)

# cleaned_quali_results.isna().sum()

In [307]:
qualifying = cleaned_quali_results

In [308]:
qualifying[qualifying.raceId == 18]

,driverId,constructorId,q1_position,q2_position,q3_position,raceId
0,1,1,10.0,1.0,1.0,18
1,9,2,4.0,2.0,2.0,18
2,5,1,1.0,3.0,3.0,18
3,13,6,3.0,5.0,4.0,18
4,2,2,2.0,4.0,5.0,18
5,15,7,9.0,9.0,6.0,18
6,3,3,6.0,7.0,7.0,18
7,14,9,8.0,8.0,8.0,18
8,10,7,16.0,10.0,9.0,18
9,20,5,11.0,6.0,10.0,18


In [309]:
qualifying.isna().sum()

driverId         0
constructorId    0
q1_position      0
q2_position      0
q3_position      0
raceId           0
dtype: int64

In [310]:
quali_w_results = qualifying.merge(
    race_results,
    on=["raceId", "driverId", "constructorId"],
    how="left"
)

In [72]:
# quali_results = quali_results.drop(["resultId", ""])
quali_w_results[quali_w_results.raceId == 1040]

,driverId,constructorId,q1_position,q2_position,q3_position,raceId,resultId,number,grid,position,...,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
8534,1,131,2.0,4.0,1.0,1040,24808.0,44.0,1.0,3.0,...,3.0,15.0,53.0,+22.729,5663093.0,48.0,7.0,1:38.075,214.660,1.0
8535,830,9,5.0,9.0,2.0,1040,24807.0,33.0,2.0,2.0,...,2.0,18.0,53.0,+7.729,5648093.0,53.0,3.0,1:37.332,216.298,1.0
8536,822,131,1.0,2.0,3.0,1040,24806.0,77.0,3.0,1.0,...,1.0,26.0,53.0,1:34:00.364,5640364.0,51.0,1.0,1:37.030,216.972,1.0
8537,815,211,7.0,5.0,4.0,1040,24809.0,11.0,4.0,4.0,...,4.0,12.0,53.0,+30.558,5670922.0,52.0,8.0,1:38.141,214.515,1.0
8538,817,4,6.0,1.0,5.0,1040,24810.0,3.0,5.0,5.0,...,5.0,10.0,53.0,+52.065,5692429.0,53.0,6.0,1:37.886,215.074,1.0
8539,832,1,12.0,3.0,6.0,1040,24824.0,55.0,6.0,NaN,...,19.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,3.0
8540,839,4,4.0,10.0,7.0,1040,24812.0,31.0,7.0,7.0,...,7.0,6.0,53.0,+1:08.006,5708370.0,48.0,13.0,1:39.216,212.191,1.0
8541,846,1,9.0,6.0,8.0,1040,24820.0,4.0,8.0,15.0,...,15.0,0.0,52.0,NaN,NaN,50.0,5.0,1:37.377,216.198,11.0
8542,842,213,8.0,7.0,9.0,1040,24814.0,10.0,9.0,9.0,...,9.0,2.0,53.0,+1:29.766,5730130.0,50.0,2.0,1:37.231,216.523,1.0
8543,848,9,11.0,8.0,10.0,1040,24815.0,23.0,15.0,10.0,...,10.0,1.0,53.0,+1:37.860,5738224.0,51.0,9.0,1:38.377,214.001,1.0


In [311]:
quali_w_results = quali_w_results.drop(["resultId", "number", "grid", "positionText",
                      "positionOrder", "points","laps", "time",
                      "milliseconds", "fastestLap", "rank",
                      "fastestLapTime", "fastestLapSpeed", "statusId"], axis=1)

quali_w_results

,driverId,constructorId,q1_position,q2_position,q3_position,raceId,position
0,1,1,10.0,1.0,1.0,18,1.0
1,9,2,4.0,2.0,2.0,18,NaN
2,5,1,1.0,3.0,3.0,18,5.0
3,13,6,3.0,5.0,4.0,18,NaN
4,2,2,2.0,4.0,5.0,18,2.0
...,...,...,...,...,...,...,...
8745,852,213,16.0,16.0,11.0,1055,NaN
8746,8,51,17.0,16.0,11.0,1055,12.0
8747,854,210,18.0,16.0,11.0,1055,18.0
8748,849,3,19.0,16.0,11.0,1055,16.0


In [312]:
quali_w_results_clean = quali_w_results.copy()

In [313]:
quali_w_results_clean = quali_w_results_clean.groupby(["raceId"]).transform(lambda x: x.fillna(x.max() + 1))
quali_w_results_clean["raceId"] = quali_w_results["raceId"]

In [314]:
quali_w_results_clean

,driverId,constructorId,q1_position,q2_position,q3_position,position,raceId
0,1,1,10.0,1.0,1.0,1.0,18
1,9,2,4.0,2.0,2.0,9.0,18
2,5,1,1.0,3.0,3.0,5.0,18
3,13,6,3.0,5.0,4.0,9.0,18
4,2,2,2.0,4.0,5.0,2.0,18
...,...,...,...,...,...,...,...
8745,852,213,16.0,16.0,11.0,20.0,1055
8746,8,51,17.0,16.0,11.0,12.0,1055
8747,854,210,18.0,16.0,11.0,18.0,1055
8748,849,3,19.0,16.0,11.0,16.0,1055


In [315]:
quali_w_results_clean = quali_w_results_clean.rename(columns=
    {
        "position": "driver_standing"
    }
)

In [316]:
#merging in constructor results
results = quali_w_results_clean.copy()
results_w_races = results.merge(
    races,
    on="raceId",
    how="left"
)

In [317]:
results_w_races = results_w_races.drop(["round", "name", "date", "time", "url"], axis=1)
results_w_races


,driverId,constructorId,q1_position,q2_position,q3_position,driver_standing,raceId,year,circuitId
0,1,1,10.0,1.0,1.0,1.0,18,2008,1
1,9,2,4.0,2.0,2.0,9.0,18,2008,1
2,5,1,1.0,3.0,3.0,5.0,18,2008,1
3,13,6,3.0,5.0,4.0,9.0,18,2008,1
4,2,2,2.0,4.0,5.0,2.0,18,2008,1
...,...,...,...,...,...,...,...,...,...
8745,852,213,16.0,16.0,11.0,20.0,1055,2021,4
8746,8,51,17.0,16.0,11.0,12.0,1055,2021,4
8747,854,210,18.0,16.0,11.0,18.0,1055,2021,4
8748,849,3,19.0,16.0,11.0,16.0,1055,2021,4


In [318]:
constructors_clean = constructor_results.copy()
constructors_clean = constructors_clean.drop(["constructorResultsId", "status"], axis=1)

In [319]:
constructors_clean["constructor_standing"] = constructors_clean.sort_values(["points"]).groupby("raceId").cumcount() + 1
constructors_clean = constructors_clean.drop(["points"], axis=1)

In [320]:
final_data = results_w_races.merge(
    constructors_clean,
    on=["raceId", "constructorId"],
    how="left"
)

In [321]:
final_data = final_data.merge(
    drivers_clean,
    on="driverId",
    how="right"
)

final_data.columns = ["driver_id", "constructor_id",
                      "q1_position", "q2_position",
                      "q3_position", "driver_standing", "race_id",
                      "year", "circuit_id", "constructor_standing",
                      "driver_code"]

In [322]:
final_data.isna().sum()

driver_id               0
constructor_id          0
q1_position             0
q2_position             0
q3_position             0
driver_standing         0
race_id                 0
year                    0
circuit_id              0
constructor_standing    0
driver_code             0
dtype: int64

In [140]:
y1 = final_data["q1_position"]
y2 = final_data["q2_position"]
y3 = final_data["q3_position"]
X = final_data[final_data.columns.difference(["driver_standing", "q3_position", "constructor_standing"])]
